In [161]:
###################################
# Name: Mingxun Wang              #
# GTid: 903357683                 #
# CSE 6240                        #
# Retrieve Pub Firms' SEC Filings #
# Date: 3/6/2021                  #
###################################

import urllib.request
import pandas as pd
import numpy as np
from numpy import random
import os
import os.path
import urllib
import requests
import time
import math
import json
import ast
from bs4 import BeautifulSoup
import string
import re
import datetime
import importlib
from dateutil.relativedelta import relativedelta 
import pandas_market_calendars as mcal # trading days only 
from sklearn.linear_model import LinearRegression # for market model alpha and beta

### read trace 
""" Import WRDS Module """
import wrds
db=wrds.Connection()

Enter your WRDS username [pipiw]:richardw
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.


Exception during reset or similar
Traceback (most recent call last):
  File "c:\users\pipiw\anaconda3\lib\site-packages\sqlalchemy\pool\base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "c:\users\pipiw\anaconda3\lib\site-packages\sqlalchemy\pool\base.py", line 893, in _reset
    pool._dialect.do_rollback(self)
  File "c:\users\pipiw\anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 558, in do_rollback
    dbapi_connection.rollback()
psycopg2.OperationalError: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.



Loading library list...
Done


In [163]:
""" Add Personal Module Path to System """
import sys
sys.path.append(r'C:\Users\pipiw\Dropbox (GaTech)\Documents\function_code')

""" Reload the Module if It Has Been Changed """
import wang_function
importlib.reload(wang_function)
from wang_function import * 

sys.path.append(r'C:\Users\pipiw\Dropbox (GaTech)\Documents\Proj_Prof.Chava_BDC\code')
import BDC_soi_clean
importlib.reload(BDC_soi_clean)
from BDC_soi_clean import *

""" Test Function of Module """
test_function(1,2)

The sum is:  3


### Steps:
1. Get a list of public firms CIKs from interaction of CRSP and COMPUSTAT

2. Get SEC filing's "masterfile"

3. Retrieve 10k filings based on links in the masterfile, and get only public firms' filings by CIKs in step 1

## CRSP Headers (to Have Public Firms)

In [98]:
""" Function of retrieving CRSP from WRDS """
def wrds_sql_CRSP_mse_retrieve():
    ## set up the SQL query code

#     # convert the list into a tuple for SQL query
#     var_list_tuple = tuple(var_list)
    # all columns; dataset: CRSPQ; table: dsf; filter: year range and cusip_id
    sql_query = """
    SELECT permno, namedt, nameendt, ticker, comnam, naics, shrcd, exchcd, permco, permno, issuno, cusip
    FROM crspq.msenames  
    WHERE nameendt >= '20090101' 
    """
    # query the data
    temp = db.raw_sql(sql_query)
    return temp

In [99]:
""" search public firms """
# get all the publicly traded firm names
mse_df = wrds_sql_CRSP_mse_retrieve() 

In [100]:
""" Get unique CUSIP """
mse_df_cusip_list = mse_df['cusip'].unique().tolist()

len(mse_df_cusip_list)

13349

## Comp-CRSP Merged Data (to get list of CIKs that to be downloaded for SEC filings)

In [105]:
""" Create the COMP-CRSP Merged Dataset """ 
""" Function of retrieving COMPUSTAT from WRDS """
def wrds_sql_COMP_retrieve():
    ## set up the SQL query code

#     # convert the list into a tuple for SQL query
#     var_list_tuple = tuple(var_list)
    # all columns; dataset: CRSPQ; table: dsf; filter: year range and cusip_id
    sql_query = """
    SELECT *
    FROM comp.funda   
    WHERE fyear >= '2018' 
    """
    # query the data
    temp = db.raw_sql(sql_query)
    return temp

""" Function of retrieving quarterly COMPUSTAT from WRDS """
def wrds_sql_COMP_qtr_retrieve():
    ## set up the SQL query code

#     # convert the list into a tuple for SQL query
#     var_list_tuple = tuple(var_list)
    # all columns; dataset: CRSPQ; table: dsf; filter: year range and cusip_id
    sql_query = """
    SELECT *
    FROM comp.fundq   
    WHERE fyearq >= '2009'
    """
    # query the data
    temp = db.raw_sql(sql_query)
    return temp

""" Function of retrieving COMPUSTAT headers from WRDS """
def wrds_sql_COMP_names_retrieve():
    ## set up the SQL query code

#     # convert the list into a tuple for SQL query
#     var_list_tuple = tuple(var_list)
    # all columns; dataset: CRSPQ; table: dsf; filter: year range and cusip_id
    sql_query = """
    SELECT *
    FROM comp.names   
    WHERE year2 >= '2009'
    """
    # query the data
    temp = db.raw_sql(sql_query)
    return temp

""" Function of retrieving COMPUSTAT ratings from WRDS """
def wrds_sql_COMP_ratings_retrieve():
    ## set up the SQL query code
 
    #     # convert the list itno a tuple for SQL query
    #     var_list_tuple = tuple(var_list)
    # all columns; dataset: CRSPQ; table: dsf; filter: year range and cusip_id
    sql_query = """
    SELECT *
    FROM comp.adsprate   
    """
    # query the data
    temp = db.raw_sql(sql_query)
    return temp

""" Function of retrieving EXECCOMPUSTAT person from WRDS """
def wrds_sql_EXECCOMP_ex_header_retrieve():
    ## set up the SQL query code
 
    sql_query = """
    SELECT *
    FROM execcomp.ex_header   
    """
    # query the data
    temp = db.raw_sql(sql_query)
    return temp

# """ Function of retrieving CRSP prices from WRDS (monthly) """
# def wrds_sql_CRSP_retrieve():
#     ## set up the SQL query code

# #     # convert the list into a tuple for SQL query
# #     var_list_tuple = tuple(var_list)
#     # all columns; dataset: CRSPQ; table: dsf; filter: year range and cusip_id
#     sql_query = """
#     SELECT permco, permno, cusip, date, prc, ret, shrout, retx, hexcd
#     FROM crspq.dsf  
#     WHERE date >= '20181201' AND date < '20200601'
#     """
#     # query the data
#     temp = db.raw_sql(sql_query)
#     return temp


##### Compustat PreProcessing

In [102]:
""" Compustat handling """
# Get SIC
comp_names_df = wrds_sql_COMP_names_retrieve() # get the compustat names after 2009 for filtering SIC

In [103]:
# Get compustat accounting info
comp_df = wrds_sql_COMP_retrieve() # get the compustat after 2019

print('********** Comp-Funda Retreive **********')
print ('No. of GVKEY: ', len(comp_df['gvkey'].unique()))
print ('No. of CRSP Matches: ', len(comp_df[comp_df['cusip'].str.slice(0,8).isin(mse_df_cusip_list)]['cusip'].unique()))
print ('No. of obs: ', len(comp_df))

********** Comp-Funda Retreive **********
No. of GVKEY:  19085
No. of CRSP Matches:  11749
No. of obs:  226562


In [104]:
# add SIC to compustat funda
comp_df_merge = pd.merge(comp_df, comp_names_df[['gvkey', 'sic']],
                   left_on = ['gvkey'], right_on = ['gvkey'], 
                   how = 'inner')

print('********** Add SIC **********')
print ('No. of GVKEY: ', len(comp_df_merge['gvkey'].unique()))
print ('No. of CRSP Matches: ', len(comp_df_merge[comp_df_merge['cusip'].str.slice(0,8).isin(mse_df_cusip_list)]['cusip'].unique()))
print ('No. of obs: ', len(comp_df_merge))

********** Add SIC **********
No. of GVKEY:  19085
No. of CRSP Matches:  11749
No. of obs:  226562


In [106]:
""" Compustat filtering and processing """
# list of neccesary vars for funda
comp_var_list = ['conm', 'cik', 'at', 'lt', 'che', 'ni', 'sale', 'pstkl', 'txditc', 'pstkrv', 'seq', 'pstk', 
                 'gvkey', 'cusip', 'datadate', 'fyear', 'indfmt', 'datafmt', 'popsrc', 'consol', 'sic'] 

# # list of neccesary vars for fundq
# comp_var_list = ['conm', 'atq', 'ltq', 'che', 'ni', 'pstkq', 'txditcq', 'seqq',  
#                  'gvkey', 'cusip', 'datadate', 'fyearq', 'indfmt', 'datafmt', 'popsrc', 'consol', 'sic'] 

comp_df_2 = comp_df_merge[comp_var_list] # keep only the required cols
# get cusip8
comp_df_2['cusip'] = comp_df_2['cusip'].str.slice(0,8)

### Filtering formats
comp_df_2 = comp_df_2[(comp_df_2['indfmt'] == 'INDL')
                      &(comp_df_2['datafmt'] == 'STD')
                      &(comp_df_2['popsrc'] == 'D')
                     &(comp_df_2['consol'] == 'C')]

print('********** After Format Filter **********')
print ('No. of GVKEY: ', len(comp_df_2['gvkey'].unique()))
print ('No. of CRSP Matches: ', len(comp_df_2[comp_df_2['cusip'].isin(mse_df_cusip_list)]['cusip'].unique()))
print ('No. of obs: ', len(comp_df_2))

### Filtering SIC
# remove sic missing rows
comp_df_2 = comp_df_2[~comp_df_2['sic'].isnull()]

# remove financial firms (SIC: 6000-6999) remove utilities (SIC: 4910-4940)
comp_df_3 = comp_df_2[~((comp_df_2['sic'].astype(int).between(6000,6999, inclusive = True))|
                        (comp_df_2['sic'].astype(int).between(4910,4940, inclusive = True)))]

print('********** After SIC Filter - Removing fin and uti firms **********')
print ('No. of GVKEY: ', len(comp_df_3['gvkey'].unique()))
print ('No. of CRSP Matches: ', len(comp_df_3[comp_df_3['cusip'].isin(mse_df_cusip_list)]['cusip'].unique()))
print ('No. of obs: ', len(comp_df_3))

### Filtering by year
# get calender year
comp_df_3['year'] = pd.DatetimeIndex(comp_df_3['datadate']).year

comp_df_4 = comp_df_3[(comp_df_3['year'] >= 2015) &(comp_df_3['year'] <= 2020)]

print('********** After Year Filter **********')
print ('No. of GVKEY: ', len(comp_df_4['gvkey'].unique()))
print ('No. of CRSP Matches: ', len(comp_df_4[comp_df_4['cusip'].isin(mse_df_cusip_list)]['cusip'].unique()))
print ('No. of obs: ', len(comp_df_4))


### keep only compustat available cusips

# filter CRSP data by compustat availability
comp_df_5 = comp_df_4[comp_df_4['cusip'].isin(mse_df_cusip_list)]

print('********** Keep firms that are in CRSP also **********')
print ('No. of GVKEY: ', len(comp_df_5['gvkey'].unique()))
print ('No. of CRSP Matches: ', len(comp_df_5[comp_df_5['cusip'].isin(mse_df_cusip_list)]['cusip'].unique()))
print ('No. of obs: ', len(comp_df_5))

""" IMPORTANT: LIST OF CIK TO BE DOWNLOADED """
### get compustat available cusips
comp_df_5 = comp_df_5[~comp_df_5['cik'].isnull()]

comp_cik_list = comp_df_5['cik'].unique().astype(int).tolist()


<ipython-input-106-babce7c19763>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_df_2['cusip'] = comp_df_2['cusip'].str.slice(0,8)


********** After Format Filter **********
No. of GVKEY:  18973
No. of CRSP Matches:  11748
No. of obs:  131468
********** After SIC Filter - Removing fin and uti firms **********
No. of GVKEY:  11094
No. of CRSP Matches:  6112
No. of obs:  77607
********** After Year Filter **********
No. of GVKEY:  7838
No. of CRSP Matches:  4857
No. of obs:  35176
********** Keep firms that are in CRSP also **********
No. of GVKEY:  4857
No. of CRSP Matches:  4857
No. of obs:  23106


<ipython-input-106-babce7c19763>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_df_3['year'] = pd.DatetimeIndex(comp_df_3['datadate']).year


# RETRIEVE SEC FILINGS
## Setting up the directory

In [43]:
### Setting up the directory
current_path = os.getcwd() # get current path

## saving option 1
# parent_path = os.path.dirname(current_path) # get parent path
# path = os.path.join(os.path.dirname(current_path), 'data', 'SEC') # define download path

## saving option 2
path = os.path.join(current_path, 'data', 'SEC') # define download path

In [44]:
path

'C:\\Users\\pipiw\\Dropbox (GaTech)\\Documents\\GT_SPRING_2021\\CSE6240\\project\\data\\SEC'

In [45]:
### Function of generate lists of required years and quarters

def yrqr(startyr, endyr):
    lstquarter = ["QTR1", "QTR2", "QTR3", "QTR4"] # generate a list of quarter names
    lstyr = [year for year in range(startyr,endyr+1)] # generate a list of years
    lstyr = sorted(lstyr*4) # generate 4 duplications of years to match four quarters
    lstqtr = ["QTR1", "QTR2", "QTR3", "QTR4"] # generate a list of quarter names
    lstqtr = lstqtr*int(len(lstyr)/4) # generate years' many quarters for matching purpose
    lstyrqtr = pd.DataFrame(lstqtr, lstyr) # transform into a dataframe of all the years with their quarters
    lstyrqtr["Year"] = lstyrqtr.index # set the index values
    lstindex = [index for index in range(0, len(lstqtr))] # generate the number index
    lstyrqtr = lstyrqtr.set_index([lstindex]) # set the index to a list of numbers
    lstyrqtr.columns = ["Quarter", "Year"] # name the columns
    return lstyrqtr;
# yrqr(startyr, endyr)

### Function of generating a local name for the 8-K file

def generate_txt_name(edgarname):
    lstnewname = edgarname.split("/") # seperate the edgarname part
    newname = lstnewname[3] # get only the last .txt part
    return newname;
# edgarname = "edgar/data/200754/0000950123-95-001458.txt" # testname
# generate_txt_name(edgarname)

In [52]:
dfyrqtr

,Quarter,Year
0,QTR1,2019
1,QTR2,2019
2,QTR3,2019
3,QTR4,2019
4,QTR1,2020
5,QTR2,2020
6,QTR3,2020
7,QTR4,2020


## Download the SEC Filings

In [99]:
###### Main Function
startyr = 2020 # set start year
endyr = 2020 # set end year
dfyrqtr = yrqr(startyr, endyr) # generate all the year and quarters needed

# read list of public firms' CIKs
# comp_cik_list

# store the info of downloaded files
df_files_downloaded = pd.DataFrame()

# initialize empty var
temp_year = None
temp_qtr = None

## download all the files
for j in dfyrqtr.index:

    start_time = time.time() # calc run time per ite
    temp_year = dfyrqtr.loc[j]["Year"]
    temp_qtr = dfyrqtr.loc[j]["Quarter"]
    
    # find the master file for the corresponding year and quarter
    url = "https://www.sec.gov/Archives/edgar/full-index/{}/{}/master.idx".format(temp_year, 
                                                                                  temp_qtr)
    
    # read the master index file and keep a local copy
    master_idx = pd.read_csv(url,sep="|", engine='python', 
                           header = None, skiprows = 11,
                          names = ['CIK','CompName','FormType','DateFiled','Filename'],encoding='latin-1')
    
    # keep only the public firms info from the master index file
    # and only 10-K filings
    master_idx_pubfirms_only = master_idx[(master_idx['CIK'].isin(comp_cik_list)) & 
                                          (master_idx['FormType'] == '10-K')]
    
#     print(master_idx_pubfirms_only)
    
    # show the downloading progress
    print(temp_year, 
          temp_qtr,
         "Start...")
    
    # initialize empty var
    temp_file = None
    
    ## download all the public firms' files
    for file in master_idx_pubfirms_only.index:
#         print(file)
        
        # faster for recalling
        temp_file = master_idx_pubfirms_only.loc[file]

        # download the file from EDGAR
        file_url = "https://www.sec.gov/Archives/{}".format(temp_file['Filename'])
        filename = generate_txt_name(temp_file['Filename']) # keep the file name
#         txt_path = os.path.join(path, 
#                                 str(temp_year), 
#                                 str(temp_qtr),
#                                 filename) # generate the address based on file name 
# #         print(txt_path)
        
#         os.makedirs(os.path.dirname(txt_path), exist_ok=True) # create the path if not exists
        
#         # download the 8k into the local address
#         target_online = requests.get(file_url)
#         with open(txt_path,'wb') as f:
#             f.write(target_online.content)

        ## record download info
        # store SEC file info columns
        temp_line = pd.DataFrame()
        temp_line = temp_file[["CIK", 
                             "CompName",
                             "FormType",
                             "DateFiled"]]

        temp_line["Year"] = temp_year # add year column
        temp_line["Quarter"] = temp_qtr # add quarter column
        
#         print(temp_line)
#         print(filename)

        # add download path column
        temp_line["file_path"] = os.path.join(str(temp_year), 
                                            str(temp_qtr),
                                            filename) # generate the address based on file name 
#         temp_line["file_path"] = os.path.join(str(temp_year), 
#                                             str(temp_qtr),
# #                                             str(temp_file['CIK']), 
# #                                             temp_file['FormType'],
#                                             filename) # generate the address based on file name 
        df_files_downloaded =df_files_downloaded.append(temp_line)
#         print(txt_path)
        
#         print (temp_line)
        
    # show the downloading progress
    print(temp_year, 
          temp_qtr,
         "Done...")
    
    print('Ite time: ', (time.time() - start_time))
    
    ## store downloaded files info into a table
    df_files_downloaded = df_files_downloaded.sort_values(by=['Year','Quarter']).reset_index(drop=True) # re-index with number series

    # output the result to a .csv file
    df_files_downloaded.to_csv(os.path.join(path, "note_files_downloaded_info.csv"), sep = ",") 
    
    
    # wait 15 seconds to keep download (avoid server download limit)
    time.sleep(15)


2020 QTR1 Start...
2020 QTR1 Done...
Ite time:  16.662134885787964
2020 QTR2 Start...
2020 QTR2 Done...
Ite time:  5.256421089172363
2020 QTR3 Start...
2020 QTR3 Done...
Ite time:  4.169765472412109
2020 QTR4 Start...
2020 QTR4 Done...
Ite time:  4.7546117305755615


In [98]:
df_files_downloaded

,CIK,CompName,DateFiled,FormType,Quarter,Year,file_path
0,1000228.0,HENRY SCHEIN INC,2020-02-20,10-K,QTR1,2020.0,2020\QTR1\1000228\10-K\0001000228-20-000018.txt
1,1000229.0,CORE LABORATORIES N V,2020-02-10,10-K,QTR1,2020.0,2020\QTR1\1000229\10-K\0001564590-20-004075.txt
2,1000230.0,OPTICAL CABLE CORP,2020-01-27,10-K,QTR1,2020.0,2020\QTR1\1000230\10-K\0001437749-20-001224.txt
3,1000623.0,SCHWEITZER MAUDUIT INTERNATIONAL INC,2020-03-02,10-K,QTR1,2020.0,2020\QTR1\1000623\10-K\0001000623-20-000048.txt
4,1000694.0,NOVAVAX INC,2020-03-11,10-K,QTR1,2020.0,2020\QTR1\1000694\10-K\0001104659-20-031944.txt
5,1000697.0,WATERS CORP /DE/,2020-02-25,10-K,QTR1,2020.0,2020\QTR1\1000697\10-K\0001193125-20-048303.txt
6,1000753.0,"INSPERITY, INC.",2020-02-12,10-K,QTR1,2020.0,2020\QTR1\1000753\10-K\0001000753-20-000013.txt
7,1001082.0,DISH Network CORP,2020-02-19,10-K,QTR1,2020.0,2020\QTR1\1001082\10-K\0001558370-20-000945.txt
8,1001233.0,"SANGAMO THERAPEUTICS, INC",2020-02-28,10-K,QTR1,2020.0,2020\QTR1\1001233\10-K\0001628280-20-002582.txt
9,1001316.0,"TG THERAPEUTICS, INC.",2020-03-02,10-K,QTR1,2020.0,2020\QTR1\1001316\10-K\0001558370-20-001897.txt


# PreProcess CEO Gender and Other Information

In [107]:
""" Retrieve Executive information from EXECCOMP """
execcomp_ex = wrds_sql_EXECCOMP_ex_header_retrieve()

In [108]:
""" PreProcess execcomp_ex """
execcomp_ex_2 = execcomp_ex[execcomp_ex['pceo'] == 'CEO']

# execcomp_ex_2 = execcomp_ex[(~execcomp_ex['becameceo'].isnull())
#                             & ((~execcomp_ex['leftofc'] <= pd.to_datetime('2019-01-01')) | 
#                                (execcomp_ex['leftofc'].isnull()))]

In [79]:
execcomp_ex_2

,execrank,becameceo,joined_co,rejoin,leftofc,leftco,releft,co_per_rol,pceo,pcfo,...,tele,sicdesc,naicsdesc,inddesc,spcode,ticker,sub_tele,naics,spindex,sic
69,1.0,2018-06-01,None,None,None,None,None,51547.0,CEO,None,...,630 227 2000,"MACHINERY, EQUIPMENT, AND SUPPLIES-WHOLESALE",Transportation Equipment and Supplies (except ...,Aerospace & Defense,EX,AIR,630.0,423860,2010.0,5080.0
83,1.0,2003-08-01,1994-01-01,None,None,None,None,8741.0,CEO,None,...,952-938-8080,TELEPHONE AND TELEGRAPH APPARATUS,Telephone Apparatus Manufacturing,Communications Equipment,EX,ADCT.1,952.0,334210,4520.0,3661.0
125,2.0,2006-07-01,None,None,None,None,None,33287.0,CEO,None,...,855 424 7349,PHARMACEUTICAL PREPARATIONS,Pharmaceutical Preparation Manufacturing,Pharmaceuticals,EX,ALO.2,855.0,325412,3520.0,2834.0
150,5.0,2013-12-09,None,None,None,None,None,49256.0,CEO,None,...,817 963 1234,"AIR TRANSPORTATION, SCHEDULED",Scheduled Passenger Air Transportation,Airlines,EX,AAL,817.0,481111,2030.0,4512.0
161,1.0,1995-11-02,None,None,1996-08-26,1996-08-26,None,12614.0,CEO,None,...,714-727-4141,ELECTRONIC COMPUTERS,Electronic Computer Manufacturing,Computer Hardware,EX,ASTA,714.0,334111,4520.0,3571.0
169,2.0,2007-10-01,1989-11-01,None,None,None,None,19963.0,CEO,None,...,516-694-6700,INSTRUMENTS FOR MEAS & TESTING OF ELECTRICITY ...,"Capacitor, Resistor, Coil, Transformer, and Other",Electronic Equipment & Instruments,EX,ARXX,516.0,334416,4520.0,3825.0
188,1.0,2015-04-01,None,None,None,None,None,53405.0,CEO,None,...,864-967-2150,ELECTRONIC COMPONENTS AND ACCESSORIES,"Capacitor, Resistor, Coil, Transformer, and Other",Electronic Components,EX,AVX,864.0,334416,4520.0,3670.0
204,1.0,2009-04-30,2002-12-01,None,2019-11-15,None,None,27518.0,CEO,None,...,602-250-1000,ELECTRIC SERVICES,"Electric Power Generation, Transmission and Di...",Electric Utilities,EX,PNW,602.0,2211,5510.0,4911.0
228,1.0,2014-11-10,None,None,None,None,None,50434.0,CEO,None,...,385-351-1369,"EQUIPMENT RENTAL AND LEASING, NEC",Sales Financing,Consumer Finance,EX,PRG,385.0,522220,4020.0,7359.0
239,1.0,1999-01-01,1984-01-01,None,2020-03-31,None,None,13896.0,CEO,None,...,224 667 6100,ELECTROMEDICAL AND ELECTROTHERAPEUTIC APPARATUS,Electromedical and Electrotherapeutic Apparatu...,Health Care Equipment,EX,ABT,224.0,334510,3510.0,3845.0


In [51]:
len(execcomp_ex[(execcomp_ex['pceo'] == 'CEO') & (execcomp_ex['gender'] == 'FEMALE')])/len(execcomp_ex[execcomp_ex['pceo'] == 'CEO'])

0.04126403760773048

In [52]:
len(execcomp_ex[(execcomp_ex['pceo'] == 'CEO') & (execcomp_ex['gender'] == 'FEMALE')])

158

In [110]:
comp_df_5.columns

Index(['conm', 'cik', 'at', 'lt', 'che', 'ni', 'sale', 'pstkl', 'txditc',
       'pstkrv', 'seq', 'pstk', 'gvkey', 'cusip', 'datadate', 'fyear',
       'indfmt', 'datafmt', 'popsrc', 'consol', 'sic', 'year'],
      dtype='object')

In [117]:
comp_df_5

,conm,cik,at,lt,che,ni,sale,pstkl,txditc,pstkrv,...,gvkey,cusip,datadate,fyear,indfmt,datafmt,popsrc,consol,sic,year
8,AAR CORP,0000001750,1515.000,669.900,54.700,10.200,1594.300,0.000,104.600,0.000,...,001004,00036110,2015-05-31,2014.0,INDL,STD,D,C,5080,2015
10,AAR CORP,0000001750,1442.100,576.300,31.200,47.700,1662.600,0.000,34.300,0.000,...,001004,00036110,2016-05-31,2015.0,INDL,STD,D,C,5080,2016
12,AAR CORP,0000001750,1504.100,589.900,10.300,56.500,1767.600,0.000,37.200,0.000,...,001004,00036110,2017-05-31,2016.0,INDL,STD,D,C,5080,2017
14,AAR CORP,0000001750,1524.700,588.400,41.600,15.600,1748.300,0.000,15.700,0.000,...,001004,00036110,2018-05-31,2017.0,INDL,STD,D,C,5080,2018
16,AAR CORP,0000001750,1517.200,611.300,41.100,7.500,2051.800,0.000,0.000,0.000,...,001004,00036110,2019-05-31,2018.0,INDL,STD,D,C,5080,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226271,NOVUS CAPITAL CORPORAT,0001807707,100.438,3.078,0.312,-3.458,0.000,0.000,0.000,0.000,...,036419,67012U10,2020-12-31,2020.0,INDL,STD,D,C,9995,2020
226300,ATLAS TECHNICAL CONSULTANTS,0001751143,367.125,349.298,14.062,-11.506,468.217,NaN,NaN,NaN,...,036515,04943010,2020-12-31,2020.0,INDL,STD,D,C,8700,2020
226321,OPEN LENDING CORP,0001806201,294.009,267.387,104.148,-97.564,108.892,NaN,0.000,NaN,...,036598,68373J10,2020-12-31,2020.0,INDL,STD,D,C,7372,2020
226484,MINISO GR HOL LTD -ADR,0001815846,826.182,534.812,405.009,-37.127,1271.068,337.101,0.000,337.101,...,037157,66981J10,2020-06-30,2020.0,INDL,STD,D,C,5331,2020


In [109]:
""" Read the SEC filings download """
sec_fil = pd.read_csv(r'C:\Users\pipiw\Dropbox (GaTech)\Documents\GT_SPRING_2021\CSE6240\CSE 6240 Project\data\SEC\note_files_downloaded_info.csv',
                     index_col = ['Unnamed: 0']) # use the unnamed index as the index avoiding redundant cols

In [120]:
""" Add gvkey to sec filings df """
# get the last record reported in fiscal year 2019
comp_df_6 = comp_df_5.copy()
comp_df_6 = comp_df_6[comp_df_6['fyear'] ==2019].groupby(['cik']).tail(1)

# get unique cik and gvkey match
cik_gvkey_mat = comp_df_6[['cik', 'gvkey', 'cusip', 'at', 'lt', 'che', 'ni', 'sale']].drop_duplicates()

cik_gvkey_mat['cik'] = cik_gvkey_mat['cik'].astype(float)

### merge sec_fil with comp_df to get gvkey
sec_fil_2 = pd.merge(sec_fil, cik_gvkey_mat,
                   left_on = ['CIK'], right_on = ['cik'], 
                   how = 'inner')

print('********** Add gvkey and cusip based on cik to sec filings df **********')
print ('No. of GVKEY: ', len(sec_fil_2['gvkey'].unique()))
print ('No. of CUSIP: ', len(sec_fil_2['cusip'].unique()))
print ('No. of CIK: ', len(sec_fil_2['CIK'].unique()))

### merge execcomp data
sec_fil_3 = pd.merge(sec_fil_2, execcomp_ex_2,
                   left_on = ['cusip'], right_on = ['cusip'], 
                   how = 'inner')

print('********** Add gvkey and cusip based on cik to sec filings df **********')
print ('No. of GVKEY: ', len(sec_fil_3['gvkey_x'].unique()))
print ('No. of CUSIP: ', len(sec_fil_3['cusip'].unique()))
print ('No. of CIK: ', len(sec_fil_3['CIK'].unique()))
print ('No. of Female CEO: ', len(sec_fil_3[(sec_fil_3['gender'] == 'FEMALE')]), 'as penct ',
      "{:.2%}".format(len(sec_fil_3[(sec_fil_3['gender'] == 'FEMALE')])/len(sec_fil_3)))

********** Add gvkey and cusip based on cik to sec filings df **********
No. of GVKEY:  3009
No. of CUSIP:  3009
No. of CIK:  3009
********** Add gvkey and cusip based on cik to sec filings df **********
No. of GVKEY:  1229
No. of CUSIP:  1229
No. of CIK:  1229
No. of Female CEO:  70 as penct  5.68%


In [87]:
len(sec_fil_3[(sec_fil_3['gender'] == 'FEMALE')])/len(sec_fil_3)

0.056588520614389654

In [122]:
""" output the file """
sec_fil_3.to_csv(r'C:\Users\pipiw\Dropbox (GaTech)\Documents\GT_SPRING_2021\CSE6240\CSE 6240 Project\data\ceo_gender_file_ver1.csv')

In [121]:
sec_fil_3

,CIK,CompName,DateFiled,FormType,Quarter,Year,file_path,cik,gvkey_x,cusip,...,tele,sicdesc,naicsdesc,inddesc,spcode,ticker,sub_tele,naics,spindex,sic
0,1000228.0,HENRY SCHEIN INC,2020-02-20,10-K,QTR1,2020.0,2020\QTR1\0001000228-20-000018.txt,1000228.0,061494,80640710,...,631 843 5500,"MEDICAL, DENTAL, AND HOSPITAL EQUIPMENT AND SU...","Medical, Dental, and Hospital Equipment and Su...",Health Care Distributors,EX,HSIC,631.0,423450,3510.0,5047.0
1,1000229.0,CORE LABORATORIES N V,2020-02-10,10-K,QTR1,2020.0,2020\QTR1\0001564590-20-004075.txt,1000229.0,061759,N2271710,...,31 20 420 3191,"OIL AND GAS FIELD SERVICES, NEC",Support Activities for Oil and Gas Operations,Oil & Gas Equipment & Services,EX,CLB,31.0,213112,1010.0,1389.0
2,1000623.0,SCHWEITZER MAUDUIT INTERNATIONAL INC,2020-03-02,10-K,QTR1,2020.0,2020\QTR1\0001000623-20-000048.txt,1000623.0,061519,80854110,...,800 514 0186,PAPER MILLS,Paper (except Newsprint) Mills,Paper Products,EX,SWM,800.0,322121,1510.0,2621.0
3,1000697.0,WATERS CORP /DE/,2020-02-25,10-K,QTR1,2020.0,2020\QTR1\0001193125-20-048303.txt,1000697.0,061574,94184810,...,508 478 2000,LABORATORY ANALYTICAL INSTRUMENTS,Analytical Laboratory Instrument Manufacturing,Life Sciences Tools & Services,EX,WAT,508.0,334516,3520.0,3826.0
4,1000753.0,"INSPERITY, INC.",2020-02-12,10-K,QTR1,2020.0,2020\QTR1\0001000753-20-000013.txt,1000753.0,064302,45778Q10,...,281-358-8986,HELP SUPPLY SERVICES,Temporary Help Services,Human Resource & Employment Services,EX,NSP,281.0,561320,2020.0,7363.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,915840.0,BEAZER HOMES USA INC,2020-11-12,10-K,QTR4,2020.0,2020\QTR4\0000915840-20-000068.txt,915840.0,029755,07556Q88,...,770-829-3700,OPERATIVE BUILDERS,New Housing For-Sale Builders (eff from 6/15/...,Homebuilding,EX,BZH,770.0,236117,2520.0,1531.0
1229,923120.0,GREENBRIER COMPANIES INC,2020-10-28,10-K,QTR4,2020.0,2020\QTR4\0001564590-20-048326.txt,923120.0,030463,39365710,...,503 684 7000,RAILROAD EQUIPMENT,Railroad Rolling Stock Manufacturing,Construction Machinery & Heavy Trucks,EX,GBX,503.0,336510,2010.0,3743.0
1230,929940.0,ASPEN TECHNOLOGY INC /DE/,2020-12-09,10-K,QTR4,2020.0,2020\QTR4\0000929940-20-000069.txt,929940.0,030870,04532710,...,781 221 6400,PREPACKAGED SOFTWARE,Software Publishers,Application Software,EX,AZPN,781.0,511210,4510.0,7372.0
1231,933974.0,"Brooks Automation, Inc.",2020-11-18,10-K,QTR4,2020.0,2020\QTR4\0001558370-20-013971.txt,933974.0,031477,11434010,...,978 262 2400,"SPECIAL INDUSTRY MACHINERY, NEC",Semiconductor Machinery Manufacturing,Semiconductor Equipment,EX,BRKS,978.0,333242,4530.0,3559.0


# PreProcess CRSP

In [123]:
sec_fil_3 = pd.read_csv(r'C:\Users\pipiw\Dropbox (GaTech)\Documents\GT_SPRING_2021\CSE6240\CSE 6240 Project\data\ceo_gender_file_ver1.csv',
                       index_col = ['Unnamed: 0'])

In [114]:
""" Function of retrieving CRSP from WRDS (monthly) """
def wrds_sql_CRSP_retrieve():
    ## set up the SQL query code

#     # convert the list into a tuple for SQL query
#     var_list_tuple = tuple(var_list)
    # all columns; dataset: CRSPQ; table: dsf; filter: year range and cusip_id
    sql_query = """
    SELECT permco, permno, cusip, date, prc, ret, shrout, retx, hexcd
    FROM crspq.dsf  
    WHERE date >= '20200101' AND date < '20201231'
    """
    # query the data
    temp = db.raw_sql(sql_query)
    return temp

In [115]:
""" CRSP Data """
crsp_df = wrds_sql_CRSP_retrieve()

In [124]:
""" CRSP Handling """
# create a copy of the original data
crsp_df2 = crsp_df.copy()
# convert negative stock price to positive
crsp_df2['prc'] = crsp_df2['prc'].abs()

# # remove records if missing retx
# crsp_df2 = crsp_df2[~crsp_df2['retx'].isnull()]

print('********** CRSP keep absolute, calc market cap, remove missing **********')
print ('No. of cusip: ', len(crsp_df2['cusip'].unique()))
print ('No. of obs: ', len(crsp_df2))

### keep only the needed firms
crsp_df3 = crsp_df2.copy()

crsp_df3 = crsp_df3[crsp_df3['cusip'].isin(sec_fil_3['cusip'])]

print('********** CRSP keep gender available firms **********')
print ('No. of cusip: ', len(crsp_df3['cusip'].unique()))
print ('No. of obs: ', len(crsp_df3))

### calculate returns
## get stock prices for necessary dates
list_cols = ['cusip', 'prc', 'shrout'] # set list of necessary cols from crsp
pr_0220 = crsp_df3.loc[crsp_df3[crsp_df3['date'] == pd.to_datetime('2020-02-20')].index, list_cols] # get the prices on the date
pr_0220 = pr_0220.rename(columns={"prc": "prc_0220", "shrout": "shrout_0220"}) # rename the cols for merge

pr_0325 = crsp_df3.loc[crsp_df3[crsp_df3['date'] == pd.to_datetime('2020-03-25')].index, list_cols] # get the prices on the date
pr_0325 = pr_0325.rename(columns={"prc": "prc_0325", "shrout": "shrout_0325"}) # rename the cols for merge

pr_0630 = crsp_df3.loc[crsp_df3[crsp_df3['date'] == pd.to_datetime('2020-06-30')].index, list_cols] # get the prices on the date
pr_0630 = pr_0630.rename(columns={"prc": "prc_0630", "shrout": "shrout_0630"}) # rename the cols for merge

pr_0930 = crsp_df3.loc[crsp_df3[crsp_df3['date'] == pd.to_datetime('2020-09-30')].index, list_cols] # get the prices on the date
pr_0930 = pr_0930.rename(columns={"prc": "prc_0930", "shrout": "shrout_0930"}) # rename the cols for merge

pr_1230 = crsp_df3.loc[crsp_df3[crsp_df3['date'] == pd.to_datetime('2020-12-30')].index, list_cols] # get the prices on the date
pr_1230 = pr_1230.rename(columns={"prc": "prc_1230", "shrout": "shrout_1230"}) # rename the cols for merge


## merge the selected days' prices to the main file
pr_df = pr_0220.copy() # initialize the price df
# add 0325 prices to the file
pr_df = pd.merge(pr_df, pr_0325,
                   left_on = 'cusip', right_on = 'cusip',
                   how = 'inner')
# add 0630 prices to the file
pr_df = pd.merge(pr_df, pr_0630,
                   left_on = 'cusip', right_on = 'cusip',
                   how = 'inner')
# add 0930 prices to the file
pr_df = pd.merge(pr_df, pr_0930,
                   left_on = 'cusip', right_on = 'cusip',
                   how = 'inner')
# add 1230 prices to the file
pr_df = pd.merge(pr_df, pr_1230,
                   left_on = 'cusip', right_on = 'cusip',
                   how = 'inner')

## calculate the returns
pr_df['ret_0220_0325']  = (pr_df['prc_0325'] - pr_df['prc_0220'])/pr_df['prc_0220']
pr_df['ret_0325_0630']  = (pr_df['prc_0630'] - pr_df['prc_0325'])/pr_df['prc_0325']
pr_df['ret_0220_0630']  = (pr_df['prc_0630'] - pr_df['prc_0220'])/pr_df['prc_0220']
pr_df['ret_0220_0930']  = (pr_df['prc_0930'] - pr_df['prc_0220'])/pr_df['prc_0220']
pr_df['ret_0220_1230']  = (pr_df['prc_1230'] - pr_df['prc_0220'])/pr_df['prc_0220']

## merge the prices into the main file
sec_fil_4 = sec_fil_3.copy()

sec_fil_4 = pd.merge(sec_fil_4, pr_df,
                   left_on = 'cusip', right_on = 'cusip',
                   how = 'left')

print('********** CRSP after merge the stock price returns **********')
print ('No. of cusip: ', len(sec_fil_4['cusip'].unique()))
print ('No. of obs: ', len(sec_fil_4))

## add FF48 industry code 
sec_fil_5 = sec_fil_4.copy() # create a copy
# read the match file
sic_ff_match_path = r'C:\Users\pipiw\Dropbox (GaTech)\Documents\Datasets\FF_industry\SIC_to_Fama_French_industry.csv'
sic_ff_match = pd.read_csv(sic_ff_match_path)

# # convert sic to int for matching
# sec_fil_5['sic0'] = sec_fil_5['sic'].astype(int)

# match sic to FF48
sec_fil_5 = pd.merge(sec_fil_5, sic_ff_match[['SIC0', 'FF_48']],
                   left_on = 'sic', right_on = 'SIC0',
                   how = 'inner')

print('********** CRSP after add FF48 industry code **********')
print ('No. of cusip: ', len(sec_fil_5['cusip'].unique()))
print ('No. of obs: ', len(sec_fil_5))

********** CRSP keep absolute, calc market cap, remove missing **********
No. of cusip:  8423
No. of obs:  1932596
********** CRSP keep gender available firms **********
No. of cusip:  1219
No. of obs:  301523
********** CRSP after merge the stock price returns **********
No. of cusip:  1229
No. of obs:  1233


In [136]:
""" output the file """
sec_fil_5.to_csv(r'C:\Users\pipiw\Dropbox (GaTech)\Documents\GT_SPRING_2021\CSE6240\CSE 6240 Project\data\ceo_gender_file.csv')

In [143]:
sec_fil_3.columns

Index(['CIK', 'CompName', 'DateFiled', 'FormType', 'Quarter', 'Year',
       'file_path', 'cik', 'gvkey_x', 'cusip', 'at', 'lt', 'che', 'ni', 'sale',
       'execrank', 'becameceo', 'joined_co', 'rejoin', 'leftofc', 'leftco',
       'releft', 'co_per_rol', 'pceo', 'pcfo', 'title', 'reason', 'gvkey_y',
       'execid', 'exec_lname', 'exec_fname', 'exec_mname', 'exec_fullname',
       'gender', 'nameprefix', 'page', 'coname', 'exchange', 'address', 'city',
       'state', 'zip', 'tele', 'sicdesc', 'naicsdesc', 'inddesc', 'spcode',
       'ticker', 'sub_tele', 'naics', 'spindex', 'sic'],
      dtype='object')

# SEPERATE TASK: Create a CIK - folder name file

In [86]:
""" Read the download index """
file_dl_df = pd.read_csv(r'C:\Users\pipiw\Dropbox (GaTech)\Documents\GT_SPRING_2021\CSE6240\CSE 6240 Project\data\SEC\note_files_downloaded_info.csv',
                        index_col = ['Unnamed: 0'])

In [95]:
""" Get CIK and folder name pair """
file_dl_df2 = file_dl_df.copy()

# process the file_path col
file_dl_df2['folder_name'] = file_dl_df2['file_path'].apply(lambda x: x[x.rindex('\\') + 1: x.rindex('.')])

# process the file_path col
file_dl_df2['folder_path'] = file_dl_df2['file_path'].apply(lambda x: x[x.find('\\') + 1: x.rindex('.')])

# keep only the CIK and folder cols
file_dl_df3 = file_dl_df2[['CIK', 'folder_name', 'folder_path']]

# output the pairs
file_dl_df3.to_csv(r'C:\Users\pipiw\Dropbox (GaTech)\Documents\GT_SPRING_2021\CSE6240\CSE 6240 Project\data\SEC\CIK_folder.csv')

In [96]:
file_dl_df3

,CIK,folder_name,folder_path
0,1000228.0,0001000228-20-000018,QTR1\0001000228-20-000018
1,1000229.0,0001564590-20-004075,QTR1\0001564590-20-004075
2,1000230.0,0001437749-20-001224,QTR1\0001437749-20-001224
3,1000623.0,0001000623-20-000048,QTR1\0001000623-20-000048
4,1000694.0,0001104659-20-031944,QTR1\0001104659-20-031944
...,...,...,...
3027,929940.0,0000929940-20-000069,QTR4\0000929940-20-000069
3028,933974.0,0001558370-20-013971,QTR4\0001558370-20-013971
3029,935419.0,0001493152-20-023525,QTR4\0001493152-20-023525
3030,936395.0,0000936395-20-000042,QTR4\0000936395-20-000042


# Create Training Dataset

In [200]:
""" merge folder info to the data """
## merge folder info 
sec_fil_6 = sec_fil_5.copy()

sec_fil_6 = pd.merge(sec_fil_6, file_dl_df3,
                   left_on = 'CIK', right_on = 'CIK',
                   how = 'inner')

print('********** Gender data: after merging folder into **********')
print ('No. of cusip: ', len(sec_fil_6['cusip'].unique()))
print ('No. of obs: ', len(sec_fil_6))

## Get a list of cols for training
list_train_cols = ['CIK', 'cusip', 'CompName', 'folder_name', 'folder_path', 'at', 'lt', 'che', 'ni', 'sale',
                   'execrank', 'becameceo', 'gender','exchange','state', 'zip','FF_48', 
                   'prc_0220', 'shrout_0220', 'prc_0325', 'shrout_0325', 'prc_0630', 'shrout_0630',
                   'prc_0930', 'shrout_0930', 'prc_1230', 'shrout_1230', 'ret_0220_0325',
                   'ret_0325_0630', 'ret_0220_0630', 'ret_0220_0930', 'ret_0220_1230']
sec_fil_7 = sec_fil_6.copy()
sec_fil_7 = sec_fil_7[list_train_cols]

print('********** Gender data: create dataset for training **********')
print ('No. of cusip: ', len(sec_fil_7['cusip'].unique()))
print ('No. of obs: ', len(sec_fil_7))


""" create ids for different cols to be controls in regression """
sec_fil_8 = sec_fil_7.copy() 
sec_fil_8['exchange_id'] = sec_fil_8.groupby(sec_fil_8['exchange']).ngroup() # convert exchange to ids
sec_fil_8['gender_id'] = sec_fil_8.groupby(sec_fil_8['gender']).ngroup() # convert gender to numerical ids
sec_fil_8['state_id'] = sec_fil_8.groupby(sec_fil_8['state']).ngroup() # convert state to numerical ids

# assign value 0 to non-int zip code
sec_fil_8['zip_new'] = 0
sec_fil_8.loc[find_non_empty_digit_row(sec_fil_8, 'zip'), ['zip_new']] = sec_fil_8.loc[find_non_empty_digit_row(sec_fil_8, 'zip')]['zip']

# change the origianl zip col to the ideal format
sec_fil_8['zip'] = sec_fil_8['zip_new'].astype(int)

# drop the intermediate zip col
sec_fil_8 = sec_fil_8.drop(columns=['zip_new'])

# calculate days of being the CEO
last_date = pd.to_datetime({'year':[2020], 'month':[12], 'day':[31]})[0] 
sec_fil_8['days_becameceo'] = (last_date - sec_fil_8['becameceo'].astype('datetime64[ns]'))/datetime.timedelta(days=1)

print('********** Gender data: after converting non-numerical var and calc days of becaming CEOs **********')
print ('No. of cusip: ', len(sec_fil_8['cusip'].unique()))
print ('No. of obs: ', len(sec_fil_8))


# output the pairs
sec_fil_8.to_csv(r'C:\Users\pipiw\Dropbox (GaTech)\Documents\GT_SPRING_2021\CSE6240\CSE 6240 Project\data\ceo_gender_training.csv')

********** Gender data: after merging folder into **********
No. of cusip:  1229
No. of obs:  1233
********** Gender data: create dataset for training **********
No. of cusip:  1229
No. of obs:  1233
********** Gender data: after converting non-numerical var and calc days of becaming CEOs **********
No. of cusip:  1229
No. of obs:  1233


C:\Users\pipiw\Dropbox (GaTech)\Documents\function_code\wang_function.py:496: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ref_df[col] = ref_df[col].astype(str).str.replace('—', '0').str.replace('–', '0').str.replace('-', '0').str.replace('(', '')\
C:\Users\pipiw\Dropbox (GaTech)\Documents\function_code\wang_function.py:496: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ref_df[col] = ref_df[col].astype(str).str.replace('—', '0').str.replace('–', '0').str.replace('-', '0').str.replace('(', '')\


In [207]:
for col in sec_fil_8.columns:
    print(col, len(sec_fil_8[col].dropna()))

CIK 1233
cusip 1233
CompName 1233
folder_name 1233
folder_path 1233
at 1233
lt 1230
che 1233
ni 1233
sale 1233
execrank 1233
becameceo 1221
gender 1233
exchange 1233
state 1197
zip 1233
FF_48 1233
prc_0220 1178
shrout_0220 1178
prc_0325 1178
shrout_0325 1178
prc_0630 1178
shrout_0630 1178
prc_0930 1178
shrout_0930 1178
prc_1230 1178
shrout_1230 1178
ret_0220_0325 1178
ret_0325_0630 1178
ret_0220_0630 1178
ret_0220_0930 1178
ret_0220_1230 1178
exchange_id 1233
gender_id 1233
state_id 1233
days_becameceo 1221
